In [ ]:
import pandas as pd


# CSV 로드
df = pd.read_csv("../data/Mcdelivery_menu_prices_Kacl.csv")
df.columns = df.columns.str.strip()
df['칼로리(Kcal)'] = pd.to_numeric(df['칼로리(Kcal)'], errors='coerce')

# 사용자 입력
gender = input("성별을 입력하세요 (남/여): ").strip()
height = float(input("신장(cm): "))
weight = float(input("체중(kg): "))
set_choice = input("버거를 어떤 방식으로 추천받고 싶나요? (단품/세트): ").strip()

# 기초 대사량 계산
if gender == "남":
    bmr = 10 * weight + 6.25 * height - 5 * 25 + 5
else:
    bmr = 10 * weight + 6.25 * height - 5 * 25 - 161

total_kcal = round(bmr * 1.2)
print(f"\n📊 일일 권장 섭취량은 약 {total_kcal} kcal 입니다.")

# 버거 필터링
burger_df = df[df['카테고리'].str.contains("버거")].copy()

if set_choice == "단품":
    burger_df['칼로리당_가격'] = burger_df['가격'] / burger_df['칼로리(Kcal)']
    burger_df_sorted = burger_df.sort_values('칼로리당_가격')
    best = burger_df_sorted.iloc[0]

    print("\n🍔 단품 기준 가성비 최고의 버거 추천:")
    print(f"- 메뉴: {best['메뉴']}")
    print(f"- 가격: {best['가격']}원")
    print(f"- 칼로리: {best['칼로리(Kcal)']} kcal")
    print(f"- 칼로리당 가격: {best['칼로리당_가격']:.2f} 원/kcal")

else:
    fries = df[df['메뉴'].str.contains('스낵', na=False)].iloc[0]
    drink = df[df['카테고리'].str.contains('음료')].iloc[0]

    burger_df['총칼로리'] = (
        burger_df['칼로리(Kcal)'] + fries['칼로리(Kcal)'] + drink['칼로리(Kcal)']
    )
    burger_df['총가격'] = (
        burger_df['가격'] + fries['가격'] + drink['가격']
    )
    burger_df['칼로리당_가격'] = burger_df['총가격'] / burger_df['총칼로리']

    burger_df_sorted = burger_df.sort_values('칼로리당_가격')
    best = burger_df_sorted.iloc[0]

    print("\n🍟 세트 기준 가성비 최고의 버거 추천:")
    print(f"- 메뉴: {best['메뉴']} + {fries['메뉴']} + {drink['메뉴']}")
    print(f"- 총 가격: {best['총가격']}원")
    print(f"- 총 칼로리: {best['총칼로리']} kcal")
    print(f"- 칼로리당 가격: {best['칼로리당_가격']:.2f} 원/kcal")
